# Blackbody Simulation

This is a simple first run through creating a blackbody curve that students can change the temperature of.

In [ ]:
# Author: Andrew Louwagie Gordon
# Date Created: 29May2018
# Last Modified: 01Jun2018

In [ ]:
# Import Block
# Import the necessary packages
import ipywidgets as widgets
import bqplot as bq
import numpy as np
import tempNcolor as tc
import expToLaTeX as e2l

In [ ]:
# Function Definitions Block

def Wein(T):
    '''
    This is Wein's Law and returns the peak wavelength.
    '''
    lamda_max = 0.002897755 / T
    return lamda_max

def blackbody(lamda, T):
    '''
    This function takes the array of wavelengths and the temmperature from the slider and returns an array of fluxes that
    correspond to the blackbody curve.
    '''
    return ( (2 * h * (c ** 2)) / (lamda ** 5) ) / ( np.exp((h * c) / (lamda * k * T)) - 1 )

def cr(change=None):
    '''
    This function updates the first figure which compares the spectra of several stars.
    '''
    global wavelengths
    line4.y = [blackbody(wavelengths,Temp.value)]
    pw = Wein(Temp.value)
    list = e2l.exp2LaTeX(pw) # Get LaTeX form of number
    peak_wavelength.value =  r'\({}\)'.format(list[1])
    
def fini(change=None):
    '''
    This function updates the second figure which shows the peak of the function with respect to the visible spectrum.
    '''
    global my_wl, wide_line
    my_f = blackbody(wavelengths,Temp.value) 
    y_zeros = np.zeros_like(wavelengths)
    y_array = np.array([y_zeros, my_f])
    fin_y_array = y_array.transpose()    
    wide_line.y = [fin_y_array]     

In [ ]:
# Variable Definition Block
# Define the constants
c = 3e8
h = 6.626e-34
k = 1.38e-23

# Define the peak wavelength to be updated later
pw = 10 ** -6

# Define all wavelengths from 1 nm to 1600 nm
wavelengths = np.arange(1.0e-9, 1601.0e-9, 1e-9)

list = e2l.exp2LaTeX(pw) # Make a list from the number2LaTeX converter being used

In [ ]:
# Widget Definitions Block
# This widget controls the temperature of the black body
Temp = widgets.FloatSlider(
    min=2800,
    max=42000,
    step=100,
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

# Widget to report updated peak wavelength 
peak_wavelength = widgets.Label(
    value = r'\({}\)'.format(list[1]),
    placeholder = 'Type something',
    disabled = True   
)

In [ ]:
# Required Data Block
# Update peak wavelength
pw = Wein(Temp.value) 

# Useful arrays are defined here for reference later
f1 = blackbody(wavelengths,7000)
f2 = blackbody(wavelengths,5800)
f3 = blackbody(wavelengths,4000)
f4 = blackbody(wavelengths,Temp.value)
my_f = blackbody(wavelengths,Temp.value)

In [ ]:
# Figure 1 Definition Block
# Define the scale
x_sc = bq.LinearScale()
y_sc = bq.LinearScale()

# Define the axes
ax_x = bq.Axis(label='Wavelength (nm)', scale=x_sc, tick_format='0.0e-9f')
ax_y = bq.Axis(label='Flux', scale=y_sc, orientation='vertical', tick_format='0.0e9f')

# Define the Markers
line1= bq.Lines(x=wavelengths, # Marker 1
             y=f1,
             name = ['4000'],
             scales={'x': x_sc, 'y': y_sc},
             colors=['yellow'])

line2= bq.Lines(x=wavelengths, # Marker 2
             y=f2,
             scales={'x': x_sc, 'y': y_sc},
             colors=['orange'])

line3= bq.Lines(x=wavelengths, # Marker 3
             y=f3,
             scales={'x': x_sc, 'y': y_sc},
             colors=['red'])

line4= bq.Lines(x=wavelengths, # Marker 4, this is the interactive marker
             y=f4,
             scales={'x': x_sc, 'y': y_sc},
             colors=['blue'])

# Call the function to update the figure in real time
Temp.observe(cr, names=['value'])

# Define Figure 1
fig = bq.Figure(title = 'Blackbody Compared to Common Stars',axes=[ax_x, ax_y], animation = 1000, 
                marks=[line1, line2, line3, line4])

In [ ]:
# Figure 2 Definition Block
# Define the scale
x_sc2 = bq.LinearScale()
y_sc2 = bq.LinearScale()

# Define the axes
ax_x2 = bq.Axis(label='Wavelength (nm)', scale=x_sc, tick_format='0.0e-9f')
ax_y2 = bq.Axis(label='Flux', scale=y_sc, orientation='vertical', tick_format='0.0e9f')

# These lines define the colors to be plotted and which wavelengths they are plotted at, uses hexidecimal designation
colors_array = tc.wav2hex(wavelengths*10**9)
colors_list = colors_array.tolist()

# The curve for this figureis drawn by drawing a bunch of vertical lines that go from zero to the blackbody curve, These 
#     arrays provide the proper pairs of points that define each individual line
x_array = np.array([wavelengths, wavelengths])
fin_x_array = x_array.transpose() # Arrays must be transposed to get pairs of numbers
y_zeros = np.zeros_like(wavelengths)
y_array = np.array([y_zeros, my_f])
fin_y_array = y_array.transpose() # Arrays must be transposed to get pairs of numbers

# This is the line command that draws all the lines
wide_line = bq.Lines(x = fin_x_array, y = fin_y_array, scales={'x': x_sc2, 'y': y_sc2}, colors=colors_list)

# Implement the update function
Temp.observe(fini, names=['value'])

# Define Figure 2
fig2 = bq.Figure(title = 'Blackbody Peak Wavelength', axes=[ax_x2, ax_y2], animation = 1000, marks = [wide_line])

In [ ]:
# Display Block
# Organize the widgets to display in a presentable fashion 
top = widgets.HBox([fig,fig2]) # Make the visual figures appear on the top for eay veiwing
top.children[0].layout.width = '450px' # Resize Figure 1 so that both figures fit on the screen
top.children[1].layout.width = '450px' # Resize Figure 2 so that both figures fit on the screen
temp_slide = widgets.HBox([widgets.Label('Temperature (K)'),Temp]) # Label the temperature slider to prevent text cut-off
p_wave = widgets.HBox([widgets.Label('Peak Wavelength (m)'), peak_wavelength])
p_wave.children[1].layout.width = '100px' # Make the widget large enough to handle the LaTeX notation 
bottom = widgets.HBox([temp_slide,p_wave])
box = widgets.VBox([top, bottom], layout = widgets.Layout(align_items = 'center')) # Organize everything presentably
box # Display it